## Create AI Search Index

#### Set environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

ai_search_endpoint  = os.getenv("AI_SEARCH_ENDPOINT")
if ai_search_endpoint is None or ai_search_endpoint == "":
    print("AI_SEARCH_ENDPOINT environment variable not set.")
    exit(1)

ai_search_api_version  = os.getenv("AI_SEARCH_API_VERSION")
if ai_search_api_version is None or ai_search_api_version == "":
    print("AI_SEARCH_API_VERSION environment variable not set.")
    exit(1)

ai_search_key  = os.getenv("AI_SEARCH_KEY")
if ai_search_key is None or ai_search_key == "":
    print("AI_SEARCH_KEY environment variable not set.")
    exit(1)

grocery_index = 'raihan_grocery'
grocery_image_index = 'raihan_grocery_image'

#### Create grocery index

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(
        name="category",
        type=SearchFieldDataType.String,
        sortable=True,
        filterable=True,
    ),
    SearchableField(
        name="sub_category",
        type=SearchFieldDataType.String,
        sortable=True,
        filterable=True,
    ),
    SearchableField(
        name="product_group",
        type=SearchFieldDataType.String,
        sortable=True,
        filterable=True,
    ),
    SearchableField(
        name="product_name",
        type=SearchFieldDataType.String,
        sortable=True,
        filterable=True,
        searchable=True,
    ),
    SearchField(
        name="product_name_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="my-vector-config",
    ),
    SimpleField(name="price", type=SearchFieldDataType.Double)
]
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="my-vector-config", algorithm_configuration_name="my-algorithms-config"
        )
    ],
    algorithms=[HnswAlgorithmConfiguration(name="my-algorithms-config")],
)

index = SearchIndex(name=grocery_index, fields=fields, vector_search=vector_search)

credential = AzureKeyCredential(ai_search_key)
index_client = SearchIndexClient(ai_search_endpoint, credential)
index_client.create_index(index)

#### Create grocery image index

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(
        name="image",
        type=SearchFieldDataType.String,
        sortable=True,
        filterable=True,
    ),
    SearchField(
        name="image_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1024,
        vector_search_profile_name="my-vector-config",
    )
]
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="my-vector-config", algorithm_configuration_name="my-algorithms-config"
        )
    ],
    algorithms=[HnswAlgorithmConfiguration(name="my-algorithms-config")],
)

index = SearchIndex(name=grocery_image_index, fields=fields, vector_search=vector_search)

credential = AzureKeyCredential(ai_search_key)
index_client = SearchIndexClient(ai_search_endpoint, credential)
index_client.create_index(index)